## Языковая модель

Наша  цель-построить языковую модель, используя рекуррентную нейронную сеть. Пусть в предложении m слов. Языковая модель позволяет нам предсказывать вероятность данного предложения в виде:
 \begin{aligned} P(w_1,...,w_m) = \prod_{i=1}^{m}P(w_i \mid w_1,..., w_{i-1}) \end{aligned} 
Т.е. вероятность каждого слова предсказывается, исходя из того, какими были предыдущие слова.

Рассмотрим возможности этой модели.

Во-первых, мы можем использовать её для машинного перевода и распознавания речи: берём наиболее вероятное предложение.

Во-вторых, мы можем создавать тексты благодаря возможности выбора последующего слова на основании уже выбранных предыдущих.

**NB** В формуле условной вероятности слово $w_i$ зависит от всех предыдущих слов. В теории рекуррентные сети способны использовать "бесконечную" память, однако на практике мы столкнёмся с проблемой затухающего градиента. Поэтому количество предшествующих слов, которые будут использоваться для предсказывания следующего слова, следует ограничить.



## Предобработка текста 

Аналогичные действия рассматривались на лекции по работе с word2vec. См. https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors

Наша задача - создание модели, генерирующей новый текст. Обучающая выборка-текст. Вектор ответов нам не требуется, однако тест нужно представить в виде набора one-hot-encoded векторов.

Наши шаги:

1.Разделить текст на предложения, а предложения - на слова.

2.Составить словарь из 5000 наиболее часто встречаемых слов, остальные слова включить в словарь как  UNKNOWN_TOKEN.

3.Добавить в словарь индикаторы начала и конца предложения. Это позволит предсказывать первое слово в предложении.

4.Создать индексацию (связку) между представлением слов в виде векторов и их номером в словаре.





In [34]:
vocabulary_size = 5000 #размер словаря
unknown_token = "UNKNOWN_TOKEN" # обозначение редких слов в словаре
sentence_start_token = "SENTENCE_START" #индикатор начала предложения
sentence_end_token = "SENTENCE_END" #индикатор конца предложения
 
# Загрузите данные из файла reddit-comments-2015-08.csv
# Используя nltk.sent_tokenize, разделите каждый текст на предложения (не забудьте перевести всё в нижний регистр)  
# NB В случае возникновения проблем примените к тексту decode('utf-8')
# К каждому предложению добавьте sentence_start_token и sentence_end_token
# Ваш код ...


     
# Разбейте предложения на слова, используя nltk.word_tokenize
# Ваш код ...
 
# Для каждого слова посчитайте его частоту с помощью nltk.FreqDist
# Ваш код ...


 
# Создайте словарь, используя 5000 наиболее часто встречающихся слов, используя most_common
# Создайте связку index_to_word - просто список слов из словаря
# Все остальные слова считаем как unknown_token. Добавьте unknown_token в список index_to_word
# Создайте обратную связку word_to_index: словарь(имеется в виду тип данных) пар слово-индекс (чтобы получить такую связку используйте enumerate(index_to_word))
# Ваш код ...



 
# Замените в полученных предложениях слова, которые не вошли в словарь, на unknown_token
# Ваш код ...
 
# Создайте обучающую выборку: для каждого предложения берем все слова, кроме последнего
# Вектор ответов: все слова, кроме первого



## RNN

Создадим класс RNN с соответствующими атрибутами данных и методами. Используем Theano.

## Theano
Theano – библиотека, позволяющая определять, оптимизировать и вычислять математические выражения, особенно удобно использовать Theano для работы с  многомерным массивами. Большое достоинство Theano в том, что с помощью этой библиотеки можно автоматически дифференцировать выражения, используя chain rule. Это возможно благодаря моделированию графа вычислений http://deeplearning.net/software/theano/extending/graphstructures.html

Нейронные сети легко представляются как граф вычислений, поэтому библиотека очень удобна для работы с ними.
 
Некоторые сведения о работе с Theano:

•	Определяем векторы, скаляры, матрицы  следующим образом:

a=T.scalar('a')

b=T.vector('b')

X = T.matrix('X')


•	Для вычисления  заданных выражений используем eval:

**(X * 2).eval({X : [[1,1],[2,2]] })**
    
•	Для работы с переменными, определенными выше, нужно будет присвоить им значения.
В Theano существуют переменные shared с постоянным значением, хранящимся в памяти. 

**W1 = theano.shared(5, name='W1')**


•	Функции **func = theano.function([x], expression(x))**

То есть передаём в function аргумент-x, и то, что должна вывести функция для этого аргумента

Пример **func = theano.function([x],x*2)**

•	Если предполагается, что функция обновляет значения некоторых параметров, то конструкция следующая

**func = theano.function([input_parameters], [], updates=[(old_1, new_1), ..., (old_n,new_n)])**

•	Можно использовать разные функции из theano.tensor: **argmax, grad, T.nnet.categorical_crossentropyropy, tanh, T.nnet.softmax, также sum**


In [ ]:
import theano as theano
import theano.tensor as T

#word_dim - размерность словаря 
#hidden_dim=50 - размерность скрытого слоя
#bptt_truncate=3 - число предыдущих шагов, учитываемое при запуске механизма обратного распространения ошибки

class RNNTheano:
    
    def __init__(self, word_dim, hidden_dim=50, bptt_truncate=3):
        # переменным экземпляра класса назначаются значения, передаваемые пользователем 
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # Матрицы V,W  задайте случайными значениями из интервала (-1/sqrt(hidden_dim))
        # Матрицу U  задайте случайными значениями из интервала (-1/sqrt(word_dim))
        # Ваш код... 
        
        
        # Создайте shared variables U,V,W: 
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))

        
        # Для использования chain rule в вычислении градиента нам понадобится граф вычислений. Храним его здесь
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        U, V, W = self.U, self.V, self.W
        x = T.ivector('x')
        y = T.ivector('y')
        # вычисляем внутреннее состояние s и выход o на шаге t (формулы - см.лекции)
        def forward_prop_step(x_t, s_t_prev, U, V, W): #s_t_prev - внутреннее состояние на предыдущем шаге
            s_t = 
            o_t = 
            return 
        # Вычисление o и s в цикле http://deeplearning.net/software/theano/library/scan.html
        [o,s], updates = theano.scan(
            forward_prop_step,
            sequences=x,
            outputs_info=[None, dict(initial=T.zeros(self.hidden_dim))],
            non_sequences=[U, V, W],
            np.random.multinomial(1, next_word_probs[-1])=self.bptt_truncate,
            strict=True)
        
        #вычисляем слово, имеющее наибольшую вероятность o
        prediction = 
        #вычисляем кросс-энтропию
        o_error = 
        
        
        # вычисляем градиенты o_error по U,V,W
        dU = 
        dV = 
        dW = 
        
        # создаём функции, как это принято в Theano
        self.forward_propagation =                      #записываем в ответ только o (s не записываем)
        self.predict = 
        self.ce_error = 
        self.bptt =  #зависит от x,y и возвращает dU,dV,dW
        
        # Градиентный спуск
        # определите скалярную величину learning_rate, как это приянято в Theano
        learning_rate = 
        # матрицы U,V,W обновляем, двигаясь в направлении антиградиента
        # запишите функцию градиентного шага с updates
        self.sgd_step = theano.function([x,y,learning_rate], [], 
                      updates=   
                                       )
    #вычисляем ошибку - суммируем ce_error по всем объектам X,Y
    def calculate_total_loss(self, X, Y):
        return 
    
    #разделим calculate_total_loss на число слов
    def calculate_loss(self, X, Y):
        
  

## Обучение модели

In [ ]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=1, evaluate_loss_after=5):
    # для визуализации ошибки сохраняем её значение на каждой пятой итерации
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # если номер интерации кратен 5:вычисляем значение calculate_loss, 
        #                               записываем кортеж (число просмотренных объектов,ошибка) в список loss, 
        #                               если ошибка на последней выполненной итерации больше, чем на предпоследней, то уменьшаем learning_rate в 2 раза
        
            
            
        # для каждого объекта из выборки делаем шаг градиентного спуска и увеличиваем на 1 кол-во просмотренных объектов num_examples_seen
        for i in range(len(y_train)):
           
            

## Использование модели для генерации текста

In [ ]:
def generate_sentence(model):
    # новое предложение представляем в виде списка, добавляем в него элемент sentence_start_token из  word_to_index
    new_sentence = 
    # пока не встречаем sentence_end_token:
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs =  #вычисляем для каждого слова словаря вероятность быть следующим в нашем предложении
        sampled_word = word_to_index[unknown_token]
        # нам не нужен выбор слова,которое не попало в наш словарь
        while sampled_word == word_to_index[unknown_token]:
            # на каждом шаге t получаем на выходе вектор o вероятностей для каждого слова из слоаваря, 
            # нас интересует o на последнем шаге
            samples = np.random.multinomial(1, next_word_probs[-1]) #сэмплируем индес (получаем строку из 0 и 1)
            sampled_word = np.argmax(samples) #берём индекс, указывающий на 1
        new_sentence.append(sampled_word)# это индекс
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 10
senten_min_length = 7

for i in range(num_sentences):
    sent = []
    # хотим предложения подлиннее
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

Посмотрите, какое время занимает один градиентный шаг для объекта класса RNNTheano с 50 нейронами скрытого слоя и 5000 слов в словаре

Обучение модели даже с такими небольшими значениями параметров (vocabularu_size=5000, hidden_dim=50, bptt_truncate=3, nepoch=50) займёт много времени

Если у вас нет возможности обучить модель на всей выборке, обучите её на 100 объектах с nepoch=10 и постройте график зависимости ошибки от номера итерации. Если всё сделано верно, ошибка должна уменьшаться